In [0]:
%sql 
use postg;
DROP TABLE IF EXISTS teams;
CREATE TABLE teams 
(
       team_id       INT PRIMARY KEY,
       team_name     VARCHAR(50) NOT NULL
);


DROP TABLE IF EXISTS matches;
CREATE TABLE matches 
(
       match_id 	INT PRIMARY KEY,
       host_team 	INT,
       guest_team 	INT,
       host_goals 	INT,
       guest_goals 	INT
);

INSERT INTO teams VALUES(10, 'Give');
INSERT INTO teams VALUES(20, 'Never');
INSERT INTO teams VALUES(30, 'You');
INSERT INTO teams VALUES(40, 'Up');
INSERT INTO teams VALUES(50, 'Gonna');

INSERT INTO matches VALUES(1, 30, 20, 1, 0);
INSERT INTO matches VALUES(2, 10, 20, 1, 2);
INSERT INTO matches VALUES(3, 20, 50, 2, 2);
INSERT INTO matches VALUES(4, 10, 30, 1, 0);
INSERT INTO matches VALUES(5, 30, 50, 0, 1);



In [0]:
%sql select * from teams

In [0]:
%sql
use postg;
--SELECT * FROM teams;
with cte as (
SELECT  host_team,guest_team,
(case when host_goals > guest_goals then 3
when host_goals = guest_goals then 1
else 0 end) as host_points,
(case when guest_goals > host_goals then 3
when guest_goals = host_goals then 1
else 0 end) as guest_points
FROM matches
),
host as (select host_team, sum(host_points) as tot_points from cte group by 1),
guest as (select guest_team, sum(guest_points) as tot_points from cte group by 1),
calc as (select guest_team as team_id, sum(tot_points) as tot_score from (
select * from guest union all select * from host order by tot_points desc
)x group by 1 ) 
select team_name, coalesce(tot_score,0) as score
from teams
left join calc on teams.team_id = calc.team_id
order by score desc , team_name 





In [0]:
%sql 

with cte as (
select *,
case when host_goals > guest_goals then 3
	 when host_goals < guest_goals then 0
else 1
end host_points,
case when host_goals < guest_goals then 3
	 when host_goals > guest_goals then 0
else 1
end guest_points
from matches
),
cte1 as (
select host_team teams, host_points points
from cte
union all
select guest_team teams, guest_points points
from cte
)

select t.team_id, t.team_name, ifnull(sum(c.points), 0) total
from cte1 c right join teams t on c.teams = t.team_id
group by 1, 2
order by 3 desc, t.team_name